<a href="https://colab.research.google.com/github/kimkh415/BlackjackSimulator/blob/main/BlackjackStrategySimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# Class blackjack simulator
import numpy as np
import random


def BlackJackStrategy(hand, dealer_card):
    hard_count=sum(hand)
    cards=len(hand)
    aces_count= 1*(1 in hand)
    soft_count=0
    good_cards=[1,7,8,9,10]
    
    if cards<2:
        return 'Not enough cards'
    
    if hard_count<=11:
        soft_count= hard_count + 10*aces_count
    
    if cards==2 and soft_count==21:
        return 'Blackjack'
    
    if soft_count==20:
        return 'Stay'

    if len(hand) == 2:
        # Soft total double rules
        if soft_count==19:
            if dealer_card==6:
                return 'Double'
            else:
                return 'Stay'
        
        if soft_count==18:
            if 1 < dealer_card < 7:
                return 'Double'
            elif 6 < dealer_card < 9:
                return 'Stay'
            else:
                return 'Hit'

        if soft_count==17:
            if 2 < dealer_card < 7:
                return 'Double'
            else:
                return 'Hit'
        
        if soft_count in [15, 16]:
            if 3 < dealer_card < 7:
                return 'Double'
            else:
                return 'Hit'

        if soft_count in [13, 14]:
            if 4 < dealer_card < 7:
                return 'Double'
            else:
                return 'Hit'
        
        # Hard total double rules
        if hard_count == 11:
            return 'Double'
        
        if hard_count == 10:
            if dealer_card in [1, 10]:
                return 'Hit'
            else:
                return 'Double'
        
        if hard_count == 9:
            if 2 < dealer_card < 7:
                return 'Double'
            else:
                return 'Hit'

        # Split rules
        if hand == [1, 1]:
            return 'Split'
        
        if hand==[9,9] and dealer_card not in [1,7,10]:
            return 'Split'
        
        if hand == [8, 8]:
            return 'Split'
        
        if hand == [7, 7] and 1 < dealer_card < 8:
            return 'Split'
        
        if hand == [6, 6] and 2 < dealer_card < 7:
            return 'Split'
        
        if hand == [3, 3] or hand == [2, 2]:
            if 3 < dealer_card < 8:
                return 'Split'
    
    # Hard total rules (3+ cards at hand)
    if hard_count>=17:
        return 'Stay'

    if 12 < hard_count < 17:
        if 1 < dealer_card < 7:
            return 'Stay'
        else:
            return 'Hit'
    
    if hard_count == 12:
        if 3 < dealer_card < 7:
            return 'Stay'
        else:
            return 'Hit'
    
    if hard_count < 9:
        return 'Hit'
    
  


In [59]:
def blackjack_hand_result(bet=30, player_hand='q', dealer_cards=['q'], hard_code= 4, blackjack_bonus=1.5, verbose=False):
    dealer_card = dealer_cards[0]

    #Deck of cards, 4 types of 10
    deck=np.array([1,2,3,4,5,6,7,8,9,10,10,10,10])
    
    #hard stay condition for split aces
    hard_stay=False
    
    #Creating a random hand if none is input
    if type(player_hand)==str:
        player_hand=[random.choice(deck), random.choice(deck)]
        if verbose:
            print("---------------------------------------")
            if 1 in player_hand:
                psum = sum(player_hand)
                print(f"player hand: {player_hand} ({psum}, {psum+10})")
            else:
                print(f"player hand: {player_hand} ({sum(player_hand)})")
        
    #Turning on hard stay for split aces
    if player_hand==[1]:
        hard_stay==True
        player_hand.append(random.choice(deck))
        if verbose:
            print(f"player hand: {player_hand} ({sum(player_hand)})")
    
    #Creating dealer card, and dealer cards
    if type(dealer_card)==str:
        dealer_cards=[random.choice(deck), random.choice(deck)]
        dealer_card = dealer_cards[0]
        if verbose:
            print(f"Dealer card: {dealer_card}")

    if len(dealer_cards) == 2 and 1 in dealer_cards and sum(dealer_cards) + 10 == 21:
        if verbose:
            print(f"Dealer blackjack {dealer_cards}")
            print("lose")
        return 0 - bet

    dealer_get_more_card = False

    #If hard stay condition is false
    if hard_stay==False:
        
        # Case blackjack
        if BlackJackStrategy(player_hand, dealer_card) == 'Blackjack':
            if verbose:
                print("Player blackjack")
            
            #Seeing if casino also hit blackjack, in which case tie
            if sum(dealer_cards)==11:
                print("Dealer also blackjack")
                print("tie")
                return 0
                
            #Blackjack bonus
            if verbose:
                print("win")
            return bet * blackjack_bonus
        
        # Case not blackjack
        else:

            # Hit
            while BlackJackStrategy(player_hand, dealer_card) == 'Hit':
                #Adding one card for every hit
                player_hand.append(random.choice(deck))
                if verbose:
                    print(f'Hit ({player_hand[-1]})')

                #Player loses bet if hand goes above 21
                if sum(player_hand)>21:
                    if verbose:
                        print(f"Player bust: {player_hand} ({sum(player_hand)})")
                        print("lose")
                    return 0- bet

            # Double
            if BlackJackStrategy(player_hand, dealer_card)=='Double':
                #He gets exactly one extra card and the bet size is doubled
                player_hand.append(random.choice(deck))
                if verbose:
                    print(f'Double ({player_hand[-1]})')
                bet= bet * 2

                if sum(player_hand)>21:
                    if verbose:
                        print(f"Player bust: {player_hand} ({sum(player_hand)})")
                        print("lose")
                    return 0- bet

            # Split
            if BlackJackStrategy(player_hand, dealer_card)== 'Split':
                if verbose:
                    print('Split')
                #Runs the sim twice, as different hands
                res= blackjack_hand_result(bet, [player_hand[0]], dealer_cards) 
                res+= blackjack_hand_result(bet, [player_hand[0]], dealer_cards)
                return res
        
        #Keeps track of player's score 
        player_score=sum(player_hand)
        
        #Uses soft score if that is better for player
        if player_score<=11 and 1 in player_hand:
            player_score+=10
        
        if verbose:
            print(f"final player hand: {player_hand} ({player_score})")

        #Plays out the dealer's hand
        while True:

            #Give dealer extra card if loop hasn't broken
            if dealer_get_more_card:
              dealer_cards.append(random.choice(deck))
            
            dealer_get_more_card = True
            
            #Keep track of sum of dealer's cards
            dealer_score= sum(dealer_cards)
            
            #Keep track of soft score if dealer has an ace
            soft_score= dealer_score
            if dealer_score<=11 and 1 in dealer_cards:
                soft_score+=10
            
            #Dealer stays on all 17s
            if soft_score>=17:
                
                #If dealer bust, player wins bet
                if soft_score>21:
                    if verbose:
                        print(f"Dealer bust {dealer_cards} ({soft_score})")
                        print("win")
                    return bet
                
                #If player has more than dealer, player wins bet
                if player_score>soft_score:
                    if verbose:
                        print(f"Dealer hand {dealer_cards} ({soft_score})")
                        print("win")
                    return bet
                
                #Tie means no money changes hands
                if player_score==soft_score:
                    if verbose:
                        print(f"Dealer hand {dealer_cards} ({soft_score})")
                        print("tie")
                    return 0
                
                #If player has lower, player loses bet
                if player_score<soft_score:
                    if verbose:
                        print(f"Dealer hand {dealer_cards} ({soft_score})")
                        print("lose")
                    return 0 - bet
                

       
    # No more cards for player
    if hard_stay==True:
        player_score=sum(player_hand)
        
        #Uses soft score if that is better for player
        if player_score<=11 and 1 in player_hand:
            player_score+=10
        
        #Plays out only dealer's side, as player has to stop after 1 card
        while True:
            #Give dealer extra card if loop hasn't broken
            dealer_cards.append(random.choice(deck))
            
            #Keep track of sum of dealer's cards
            dealer_score= sum(dealer_cards)
            
            #Keep track of soft score if dealer has an ace
            soft_score= dealer_score
            if dealer_score<=11 and 1 in dealer_cards:
                soft_score+=10
            
            #Dealer stays on all 17s
            if soft_score>=17:
                
                #If dealer bust, player wins bet
                if soft_score>21:
                    if verbose:
                        print(f"Dealer bust: {dealer_cards} ({soft_score})")
                        print("win")
                    return bet
                
                #If player has more than dealer, player wins bet
                if player_score>soft_score:
                    if verbose:
                        print(f"Dealer hand {dealer_cards} ({soft_score})")
                        print("win")
                    return bet
                
                #Tie means no money changes hands
                if player_score==soft_score:
                    if verbose:
                        print(f"Dealer hand {dealer_cards} ({soft_score})")
                        print("tie")
                    return 0
                
                #If player has lower, player loses bet
                if player_score<soft_score:
                    if verbose:
                        print(f"Dealer hand {dealer_cards} ({soft_score})")
                        print("lose")
                    return 0 - bet

In [55]:
def blackjack_sim(n=50_000, player_h='q', dealer_c='q', bet=10, hard_c=4, verbose=False):
    profit=0
    res=[]
    p=player_h
    q=len(player_h)
    for i in range(n):
        profit += blackjack_hand_result(bet=bet, player_hand=p[:q], dealer_cards=[dealer_c], hard_code=hard_c, verbose=verbose)
        print("-----")
        print(profit)
    
    return profit

In [60]:
blackjack_sim(n=10, bet=30, verbose=True)

---------------------------------------
player hand: [6, 9] (15)
Dealer card: 9
Hit (8)
Player bust: [6, 9, 8] (23)
lose
-----
-30
---------------------------------------
player hand: [10, 10] (20)
Dealer card: 4
final player hand: [10, 10] (20)
Dealer bust [4, 10, 9] (23)
win
-----
0
---------------------------------------
player hand: [7, 10] (17)
Dealer card: 10
final player hand: [7, 10] (17)
Dealer bust [10, 6, 10] (26)
win
-----
30
---------------------------------------
player hand: [1, 10] (11, 21)
Dealer card: 6
Player blackjack
win
-----
75.0
---------------------------------------
player hand: [1, 4] (5, 15)
Dealer card: 6
Double (3)
final player hand: [1, 4, 3] (18)
Dealer bust [6, 3, 3, 2, 2, 9] (25)
win
-----
135.0
---------------------------------------
player hand: [6, 10] (16)
Dealer card: 4
final player hand: [6, 10] (16)
Dealer hand [4, 6, 2, 2, 3] (17)
lose
-----
105.0
---------------------------------------
player hand: [10, 10] (20)
Dealer card: 5
final player han

165.0